## Pozyskanie danych
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne pakiety

In [ ]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt

## <center>Część 2</center>
### 2.5

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"] = "projekt1-415610-f19ebf54e064.json"  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

### Potrzebna funkcja

In [ ]:
def return_df(q):
    query_job = client.query(q)
    query_result = query_job.result()
    return query_result.to_dataframe()

### 2.6

In [ ]:
query = 'select * from `bigquery-public-data.covid19_open_data.covid19_open_data` limit 10'
df = return_df(query)

### Informacje o zbiorze

In [ ]:
df.info()

## <center>Część 4</center>

In [ ]:
# count_to_del = pd.DataFrame(list(zip([df[el].notna().sum() for el in df.columns], df.columns)), columns=['count', 'names'])
# stats = count_to_del['count'].describe().drop(['count', 'min', 'max'])
# stats

### Od ilu pustych wierszy usunąć kolumnę?

In [ ]:
# colors = ['red', 'green', 'pink', 'yellow', 'orange']
# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))
# count_to_del['count'].plot(kind='line', color='blue', ax=ax[0])
# for i, el in enumerate(stats.index):
#     ax[0].axhline(stats.iloc[i], color=colors[i], linestyle='solid', label=el)
# ax[0].legend(loc='upper center')
# count_to_del['count'].plot(kind='box', ax=ax[1])

### Usunięcie kolumn zawierających za dużo wartości NaN

In [ ]:
# count_to_del = count_to_del[count_to_del['count'] > 2 * count_to_del['count'].std()]
# df = df[count_to_del['names']]

### Usunięcie wierszy zawierających wartości NaN

In [ ]:
# df.dropna(inplace=True)
# df.drop(['place_id', 'wikidata_id', 'datacommons_id', 'country_code', 'iso_3166_1_alpha_2', 'new_hospitalized_patients_age_3', 'new_hospitalized_patients_male'], axis=1, inplace=True)

### Usunięcie duplikatów

In [ ]:
# df.drop_duplicates(inplace=True, ignore_index=True)

### Czyszczenie danych

In [ ]:
# count_to_del

### Gotowe dane

In [ ]:
df.info()

In [ ]:
df.head()

### 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [ ]:
query_4_1 = 'SELECT DISTINCT country_name, population, population_male, population_female, gdp_usd, gdp_per_capita_usd, latitude, longitude, area_sq_km, infant_mortality_rate, health_expenditure_usd, human_capital_index, life_expectancy FROM `bigquery-public-data.covid19_open_data.covid19_open_data` WHERE aggregation_level = 0 ORDER BY country_name'

df_4_1 = return_df(query_4_1)

In [ ]:
df_4_1.info()

### Uzupełnienie brakujących danych o kraju na podstawie jego <i>sąsiadów</i>

In [ ]:
from haversine import haversine

def save_to_csv(dataframe, filename):
    if 'czesc4' not in os.listdir(os.getcwd()):
        os.mkdir('czesc4')
    os.chdir('czesc4')
    dataframe.to_csv(filename + '.csv') # Zapis do csv
    os.chdir('..')
    
def fill_based_on_neigh(index, num=15):
    data = df_4_1.loc[index].copy()
    df_4_1['distance'] = df_4_1.apply(lambda x: haversine((data['latitude'], data['longitude']), (x['latitude'], x['longitude'])), axis=1)
    neigh = df_4_1.sort_values(by=['distance'])
    neigh = neigh.iloc[0:num]
    cols = df_4_1.columns.difference(['latitude', 'longitude', 'country_name', 'population', 'population_male', 'population_female', 'distance'])
    for col in cols:
        df_4_1.loc[index, col] = neigh[col].mean()

def fill_4_1():
    df_4_1['distance'] = 0.0
    to_float_cols = df_4_1.columns.difference(['population', 'population_male', 'population_female', 'latitude', 'longitude', 'country_name'])
    for tfc in to_float_cols:
         df_4_1[tfc] = df_4_1[tfc].astype(np.float64)
    df_4_1['population'].fillna(df_4_1['population_male'] + df_4_1['population_female'], inplace=True)
    df_4_1['population_male'].fillna(df_4_1['population'] // 2, inplace = True)
    df_4_1['population_female'].fillna(df_4_1['population'] // 2, inplace = True)
    # df_4_1['gdp_per_capita_usd'].fillna(df_4_1['gdp_usd'] / df_4_1['gdp_per_capita_usd'], inplace=True)
    # df_4_1.dropna(subset=['latitude', 'longitude'], inplace=True)
    for index, row in df_4_1.iterrows():
        if df_4_1.loc[index].isna().sum() > 0:
            fill_based_on_neigh(index=index)
    df_4_1.drop('distance', axis=1, inplace=True)
    rest_nans = df_4_1.columns[df_4_1.isna().any()]
    for rn in rest_nans:
        df_4_1[rn].fillna(df_4_1[rn].mode()[0], inplace=True)

fill_4_1()
save_to_csv(df_4_1, 'df_4_1')

### Informacje o krajach

In [ ]:
df_4_1.info()

In [ ]:
df_4_1.head()

### 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [ ]:
query_4_2 = 'SELECT country_name, date, SUM(new_confirmed) AS total_confirmed, SUM(new_confirmed_male) AS total_confirmed_male, SUM(new_confirmed_female) AS total_confirmed_female FROM `bigquery-public-data.covid19_open_data.covid19_open_data` GROUP BY country_name, date ORDER BY country_name, date'

df_4_2 = return_df(query_4_2)

In [ ]:
df_4_2.info()

### Uzupełnienie danych dotyczących zachorowań na COVID-19

In [ ]:
def fill_4_2():
    for col in df_4_2.select_dtypes(exclude=['object', 'dbdate']).columns:
        df_4_2[col].clip(lower=0, inplace=True)
    df_4_2.fillna({'total_confirmed': df_4_2.groupby(['date'])['total_confirmed'].transform('median').fillna(0).astype(np.int64)}, inplace=True)
    mask = df_4_2['total_confirmed_male'].isna() & df_4_2['total_confirmed_female'].isna() & df_4_2['total_confirmed'].notna()
    rand = np.random.randint(1, 10, size=len(mask))
    df_4_2.loc[mask, 'total_confirmed_male'] = df_4_2['total_confirmed'] // rand
    df_4_2.loc[mask, 'total_confirmed_female'] = df_4_2['total_confirmed'] - df_4_2['total_confirmed_male']

fill_4_2()
# save_to_csv(df_4_2, 'df_4_2')

### Dzienne statystyki zachorowań na COVID-19 dla każdego kraju

In [ ]:
df_4_2.info()

In [ ]:
df_4_2.head(1000)

### 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [ ]:
# query_4_3 = 'SELECT country_name, date, SUM(new_deceased) AS daily_deceased, SUM(new_deceased_male) AS daily_deceased_male, SUM(new_deceased_female) AS daily_deceased_female FROM `bigquery-public-data.covid19_open_data.covid19_open_data` GROUP BY country_name, date ORDER BY country_name, date'
# 
# df_4_3 = return_df(query_4_3)

In [ ]:
# df_4_3.info()

### Uzupełnienie danych dotyczących śmiertelności COVID-19

### 4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

### 4.5. Zdefiniuj własny dodatkowy przypadek.